In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# ---------------------------------------------------------
# FUNCIÓN: Inspección inicial del dataset
# Objetivo: Realizar un análisis exploratorio básico para
# verificar estructura, calidad y consistencia de los datos
# ---------------------------------------------------------

def inspect_data(df):
    # Mostrar las primeras filas para entender la estructura
    print("Primeras filas del dataset:")
    print(df.head())
    
    # Mostrar información general (número de registros, tipos de datos, memoria)
    print("\nInformación general del dataset:")
    print(df.info())
    
    # Contar valores nulos por cada columna
    print("\nValores nulos por columna:")
    print(df.isnull().sum())
    
    # Contar registros duplicados
    print("\nValores duplicados en el dataset:")
    print(df.duplicated().sum())
    
    # Mostrar tipos de datos de cada columna
    print("\nTipos de datos por columna:")
    print(df.dtypes) 

# ---------------------------------------------------------
# FUNCIÓN: División del dataset
# Objetivo: Separar el dataset en entrenamiento (60%),
# validación (20%) y prueba (20%)
# ---------------------------------------------------------

def split_data(df, y_column, random_state=12345):
    
    # Separar variables independientes (X) y variable objetivo (y)
    x = df.drop(y_column, axis=1)
    y = df[y_column]
    
    # Primera división: 60% train, 40% temporal
    x_train, x_temp, y_train, y_temp = train_test_split(
        x, y, test_size=0.4, random_state=random_state
    )
    
    # Segunda división: dividir el 40% en 20% valid y 20% test
    x_valid, x_test, y_valid, y_test = train_test_split(
        x_temp, y_temp, test_size=0.5, random_state=random_state
    )
    
    # Retornar todos los subconjuntos
    return x_train, x_valid, x_test, y_train, y_valid, y_test

# ---------------------------------------------------------
# FUNCIÓN: Regresión Logística con optimización de C
# Objetivo: Encontrar el mejor hiperparámetro C que
# maximice la exactitud en el conjunto de validación
# ---------------------------------------------------------

def logistic_model(features_train, target_train, features_valid, target_valid):
    
    best_model = None      # Guardará el mejor modelo encontrado
    best_acc = 0           # Guardará la mejor exactitud
    best_c = 0             # Guardará el mejor valor de C
    
    # Búsqueda manual de hiperparámetro C
    for c in [0.01, 0.1, 1, 10, 100]:
        
        # Crear modelo con el valor actual de C
        model = LogisticRegression(C=c, max_iter=1000, random_state=12345)
        
        # Entrenar modelo
        model.fit(features_train, target_train)
        
        # Predecir en validación
        predictions = model.predict(features_valid)
        
        # Calcular exactitud
        acc = accuracy_score(target_valid, predictions)
        
        # Guardar modelo si mejora la exactitud
        if acc > best_acc:
            best_acc = acc
            best_model = model
            best_c = c
    
    return best_model, best_acc, best_c

# ---------------------------------------------------------
# FUNCIÓN: Árbol de Decisión con optimización de profundidad
# Objetivo: Encontrar la mejor profundidad (max_depth)
# ---------------------------------------------------------

def decision_tree_model(features_train, target_train, features_valid, target_valid):
    
    best_model = None
    best_acc = 0
    best_depth = 0
    
    # Búsqueda del mejor max_depth
    for depth in range(1, 21):
        
        model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        
        predictions = model.predict(features_valid)
        acc = accuracy_score(target_valid, predictions)
        
        # Guardar mejor modelo
        if acc > best_acc:
            best_acc = acc
            best_model = model
            best_depth = depth
    
    return best_model, best_acc, best_depth

# ---------------------------------------------------------
# FUNCIÓN: Random Forest con optimización de hiperparámetros
# Objetivo: Encontrar la mejor combinación de:
# - n_estimators (número de árboles)
# - max_depth (profundidad máxima)
# ---------------------------------------------------------

def random_forest_model(features_train, target_train, features_valid, target_valid):
    
    best_model = None
    best_acc = 0
    best_depth = 0
    best_est = 0
    
    # Búsqueda doble de hiperparámetros
    for est in range(10, 101, 10):      # número de árboles
        for depth in range(1, 21):      # profundidad máxima
            
            model = RandomForestClassifier(
                n_estimators=est,
                max_depth=depth,
                random_state=12345
            )
            
            model.fit(features_train, target_train)
            
            predictions = model.predict(features_valid)
            acc = accuracy_score(target_valid, predictions)
            
            # Guardar mejor combinación encontrada
            if acc > best_acc:
                best_acc = acc
                best_model = model
                best_depth = depth
                best_est = est
    
    return best_model, best_acc, best_depth, best_est

# ---------------------------------------------------------
# FUNCIÓN: Comparación de modelos
# Objetivo: Comparar los mejores modelos encontrados
# y seleccionar el que tenga mayor exactitud
# ---------------------------------------------------------

def compare_models(log_model, tree_model, rf_model,
                   log_acc, tree_acc, rf_acc):
    
    # Diccionario con nombre del modelo y su desempeño
    results = {
        "Logistic Regression": (log_model, log_acc),
        "Decision Tree": (tree_model, tree_acc),
        "Random Forest": (rf_model, rf_acc)
    }
    
    # Seleccionar el modelo con mayor exactitud
    best_name = max(results, key=lambda x: results[x][1])
    
    best_model = results[best_name][0]
    best_acc = results[best_name][1]
    
    return best_name, best_model, best_acc

In [3]:
# ---------------------------------------------------------
# FUNCIÓN: Sanity Check del Dataset
# Objetivo: Realizar una verificación rápida para asegurar
# que los datos tienen sentido antes de entrenar modelos.
# Incluye:
# - Dimensiones del dataset
# - Distribución de la variable objetivo
# - Estadísticas descriptivas de las variables numéricas
# ---------------------------------------------------------

def sanity_check_data(df, target_column):
    
    # Mostrar número de filas y columnas
    print("Dimensiones:", df.shape)
    
    # Mostrar proporción de cada clase (normalizada)
    # Permite verificar si el dataset está balanceado o desbalanceado
    print("\nDistribución del target:")
    print(df[target_column].value_counts(normalize=True))
    
    # Mostrar estadísticas descriptivas:
    # media, desviación estándar, mínimo, máximo, etc.
    # Útil para detectar valores atípicos o inconsistencias
    print("\nEstadísticas descriptivas:")
    print(df.describe())
    
# ---------------------------------------------------------
# FUNCIÓN: Sanity Check del Modelo (Baseline)
# Objetivo: Crear un modelo base que siempre prediga la
# clase mayoritaria. Esto sirve como punto de comparación.
# Si el modelo entrenado no supera este baseline,
# entonces no está aprendiendo correctamente.
# ---------------------------------------------------------

def sanity_check_model(target):
    
    # Obtener la clase más frecuente en el conjunto de datos
    majority_class = target.mode()[0]
    
    # Crear predicciones constantes (todas iguales a la clase mayoritaria)
    baseline_predictions = [majority_class] * len(target)
    
    # Calcular exactitud del baseline
    baseline_accuracy = accuracy_score(target, baseline_predictions)
    
    # Mostrar resultado
    print("Accuracy baseline:", baseline_accuracy)
    
    # Retornar exactitud para compararla con modelos entrenados
    return baseline_accuracy

In [5]:
# Carga de datos
df = pd.read_csv(r'C:\Users\ACER\Documents\Proyectos TripleTen\TripleTen_Proyecto_10\files\users_behavior.csv')
df['is_ultra'] = df['is_ultra'].astype(int)

In [6]:
# Inspeccion inicial de df
inspect_data(df)

Primeras filas del dataset:
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

Valores nulos por columna:
calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Valores duplicados en el dataset:
0

Tipos de datos por columna:
calls       float64
minutes     float

In [7]:
# Division de df
x_train, x_valid, x_test, y_train, y_valid, y_test = split_data(df,'is_ultra')

In [8]:
# Corrida de modelos
log_model, log_acc, log_c = logistic_model(x_train, y_train, x_valid, y_valid)
tree_model, tree_acc, tree_depth = decision_tree_model(x_train, y_train, x_valid, y_valid)
rf_model, rf_acc, rf_depth, rf_est = random_forest_model(x_train, y_train, x_valid, y_valid)

In [ ]:
# Comparación de modelos
print(f'log_model: {log_model}')
print(f'tree_model: {tree_model}')
print(f'rf_model: {rf_model}')

log_model: LogisticRegression(C=0.01, max_iter=1000, random_state=12345)
tree_model: DecisionTreeClassifier(max_depth=3, random_state=12345)
rf_model: RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)


In [11]:
# Comprobracion de modelos
best_name, best_model, best_valid_acc = compare_models(log_model, tree_model, rf_model, log_acc, tree_acc, rf_acc)

In [13]:
# Evaluar modelos en test
test_predictions = best_model.predict(x_test)
test_acc = accuracy_score(y_test, test_predictions)

print("Mejor modelo:", best_name)
print("Accuracy validación:", best_valid_acc)
print("Accuracy test:", test_acc)

Mejor modelo: Random Forest
Accuracy validación: 0.8087091757387247
Accuracy test: 0.7962674961119751


In [14]:
# Sanity Check
sanity_check_data(df,'is_ultra')
print()
baseline_acc = sanity_check_model(y_train)
print('\nModelo cumple con sanity check:',baseline_acc < best_valid_acc)

Dimensiones: (3214, 5)

Distribución del target:
is_ultra
0    0.693528
1    0.306472
Name: proportion, dtype: float64

Estadísticas descriptivas:
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000     0.000000
75%      82.000000   571.927500    57.000000  21424.700000     1.000000
max     244.000000  1632.060000   224.000000  49745.730000     1.000000

Accuracy baseline: 0.6924273858921162

Modelo cumple con sanity check: True
